# Cuaderno de ingesta de datos

en este bloque traeremos la información desde datos abiertos

In [0]:
# Paso 1: Descargar los datos con requests y leerlos en pandas
import requests
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

url_secop = "https://www.datos.gov.co/resource/rpmr-utcd.csv?$limit=100000"
url_men = "https://www.datos.gov.co/resource/nudc-7mev.csv?$limit=100000"


# Descargar contenido
response_secop = requests.get(url_secop)
response_men = requests.get(url_men)

# Convertir contenido a pandas usando StringIO
df_secop_pd = pd.read_csv(StringIO(response_secop.text))
df_men_pd = pd.read_csv(StringIO(response_men.text))

# Convertir pandas a Spark
df_secop = spark.createDataFrame(df_secop_pd)
df_men = spark.createDataFrame(df_men_pd)

# Mostrar en Databricks
display(df_secop)
display(df_men)
 

In [0]:
# Celda 2: Guardar los DataFrames como tablas Delta
# La función .saveAsTable() guarda los datos y registra la tabla en el Unity Catalog.
# El modo "overwrite" reemplaza la tabla si ya existe, ideal para actualizaciones.
df_secop.write.format("delta").mode("overwrite").saveAsTable("main.diplomado_datos.secop")
df_men.write.format("delta").mode("overwrite").saveAsTable("main.diplomado_datos.men_estadisticas")

print("¡Tablas guardadas exitosamente en el catálogo 'main', esquema 'diplomado_datos'!")
                        

In [0]:
# Paso 1: Descargar los datos con requests y leerlos en pandas
import requests
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession
 
spark = SparkSession.builder.getOrCreate()
 
url_secop = "https://www.datos.gov.co/resource/rpmr-utcd.csv?$limit=100000&$offset=100000"
# Descargar contenido
response_secop = requests.get(url_secop)
 
# Convertir contenido a pandas usando StringIO
df_secop_pd = pd.read_csv(StringIO(response_secop.text), delimiter=',', header=0)
 
# Convertir pandas a Spark
df_secop = spark.createDataFrame(df_secop_pd)
 
# Mostrar en Databricks
display(df_secop)

# Dataset

In [0]:
from pyspark.sql.functions import col



target_schema = spark.table("main.diplomado_datos.secop").schema

df_secop_aligned = df_secop.select(
    [col(field.name).cast(field.dataType) for field in target_schema.fields]
)


df_secop_aligned.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable("main.diplomado_datos.secop")